In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
     

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": ""}


dbutils.fs.mount(
source = "abfss://tokyo-olympic-data@tokoyoolmpicdata.dfs.core.windows.net", 
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)

Out[1]: True

In [0]:
%fs 
ls "/mnt/tokyoolymic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolymic/Bronze data/,Bronze data/,0,1726733437000
dbfs:/mnt/tokyoolymic/Silver data/,Silver data/,0,1726733450000


In [0]:
spark  

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[*, 4] 
 AppName 
 Databricks Shell

In [0]:
athletes = spark.read.option("header", "true").csv("/mnt/tokyoolymic/Bronze data/athletes.csv")
Coaches = spark.read.option("header", "true").csv("/mnt/tokyoolymic/Bronze data/Coaches.csv")
EntriesGender = spark.read.option("header", "true").csv("/mnt/tokyoolymic/Bronze data/EntriesGender.csv")
Medals = spark.read.option("header", "true").csv("/mnt/tokyoolymic/Bronze data/Medals.csv")
Teams = spark.read.option("header", "true").csv("/mnt/tokyoolymic/Bronze data/Teams.csv")

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [0]:
Coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    null|
|     

In [0]:
Coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
EntriesGender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [0]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [0]:
EntriesGender = EntriesGender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
Medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|         TeamCountry|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [0]:
Medals.printSchema()

root
 |-- Rank: string (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: string (nullable = true)
 |-- Silver: string (nullable = true)
 |-- Bronze: string (nullable = true)
 |-- Total: string (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
Medals = Medals.withColumn("Rank", col("Rank").cast(IntegerType())) \
    .withColumn("Gold", col("Gold").cast(IntegerType())) \
    .withColumn("Silver", col("Silver").cast(IntegerType())) \
    .withColumn("Bronze", col("Bronze").cast(IntegerType())) \
    .withColumn("Total", col("Total").cast(IntegerType()))

In [0]:
Medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: string (nullable = true)



In [0]:
### Another way to see the schema of a DataFrame is using = option("inferSchema", "true")
athletes = spark.read.option("header", "true").option("inferSchema", "true").csv("/mnt/tokyoolymic/Bronze data/athletes.csv")
Coaches = spark.read.option("header", "true").option("inferSchema", "true").csv("/mnt/tokyoolymic/Bronze data/Coaches.csv")
EntriesGender = spark.read.option("header", "true").option("inferSchema", "true").csv("/mnt/tokyoolymic/Bronze data/EntriesGender.csv")
Medals = spark.read.option("header", "true").option("inferSchema", "true").csv("/mnt/tokyoolymic/Bronze data/Medals.csv")
Teams = spark.read.option("header", "true").option("inferSchema", "true").csv("/mnt/tokyoolymic/Bronze data/Teams.csv")

In [0]:
Medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
# find the top country with the highest number of medals
top_gold_medals_by_country = Medals.groupBy("TeamCountry").sum("Gold").orderBy(col("sum(Gold)").desc())
#display
top_gold_medals_by_country.show()

+--------------------+---------+
|         TeamCountry|sum(Gold)|
+--------------------+---------+
|United States of ...|       39|
|People's Republic...|       38|
|               Japan|       27|
|       Great Britain|       22|
|                 ROC|       20|
|           Australia|       17|
|             Germany|       10|
|              France|       10|
|               Italy|       10|
|         Netherlands|       10|
|                Cuba|        7|
|              Canada|        7|
|              Brazil|        7|
|         New Zealand|        7|
|   Republic of Korea|        6|
|             Hungary|        6|
|              Norway|        4|
|             Jamaica|        4|
|      Czech Republic|        4|
|               Kenya|        4|
+--------------------+---------+
only showing top 20 rows



In [0]:
# calculate the averge number of entries by gender for each dicipline

averge_entries_by_gender = EntriesGender.withColumn(
    'Avg_Female',EntriesGender ["Female"] / EntriesGender["Total"]
).withColumn(
    'Avg_Male',EntriesGender ["Male"] / EntriesGender["Total"]
)

averge_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/Silver-data/athletes")

In [0]:
EntriesGender.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/Silver-data/EntriesGender")
Medals.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/Silver-data/Medals")
Teams.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/Silver-data/Teams")
Coaches.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/Silver-data/Coaches")